In [2]:
import os
data_home = "/ssd2/arthur/TREC2019/data/"
run_file = "/ssd2/arthur/insy/msmarco/data/results/dev/bm25_finetuned.run"
assert os.path.isfile(run_file)

In [10]:
from tqdm import tqdm_notebook as tqdm
import os
import gzip
import csv
import random
from collections import defaultdict
split = 'dev'
queries_file = os.path.join(data_home, f"msmarco-doc{split}-queries.tsv.gz")
lookup_file = os.path.join(data_home, "msmarco-docs-lookup.tsv.gz")
qrels_file = os.path.join(data_home, f"msmarco-doc{split}-qrels.tsv")

querystring = {}
with gzip.open(queries_file, 'rt', encoding='utf-8') as f:
    tsvreader = csv.reader(f, delimiter="\t")
    for [topicid, querystring_of_topicid] in tsvreader:
        querystring[topicid] = querystring_of_topicid
        
docoffset = {}
with gzip.open(lookup_file, 'rt', encoding='utf-8') as f:
    tsvreader = csv.reader(f, delimiter="\t")
    for [docid, _, offset] in tsvreader:
        docoffset[docid] = int(offset)
qrel = {}
with open(qrels_file, 'r', encoding='utf8') as f:
    tsvreader = csv.reader(f, delimiter="\t")
    for [topicid, _, docid, rel] in tsvreader:
        assert rel == "1"
        if topicid in qrel:
            qrel[topicid].append(docid)
        else:
            qrel[topicid] = [docid]

def getcontent(docid, f):
    """getcontent(docid, f) will get content for a given docid (a string) from filehandle f.
    The content has four tab-separated strings: docid, url, title, body.
    """

    f.seek(docoffset[docid])
    line = f.readline()
    assert line.startswith(docid + "\t"), \
        f"Looking for {docid}, found {line}"
    return line.rstrip()
docs_file = os.path.join(data_home, "msmarco-docs.tsv")
output_file = os.path.join(data_home, "bm25_bert_docs.tsv")

In [13]:
!wc -l $run_file

5191671 /ssd2/arthur/insy/msmarco/data/results/dev/bm25_finetuned.run


In [14]:
output_line_format = "{}-{}\t{}\t{}\t{}\n" #query_id, document_id, querystring, document text
with open(run_file, 'r') as inf, open(docs_file, 'r') as f, open(output_file, 'w', encoding='utf-8') as outf:
    for counter, line in tqdm(enumerate(inf), total=5191671):
        [topic_id, _, doc_id, ranking, score, _] = line.split()
        is_relevant = doc_id in qrel[topic_id]
        
        query = querystring[topic_id]
        document = getcontent(doc_id, f)
        text_to_keep = ' '.join(document.split("\t")[1:])
        outf.write(output_line_format.format(topic_id, doc_id, query, text_to_keep, int(is_relevant)))